# Crypto data exploration
The question we want to answer is whether or not cryptocurrencies are actually "digital gold" and find what asset classes cryptos are actually most correlated with.

In this notebook, we run a correlation of cryptos with various assets - including SPY, gold ETFs, and individual stocks (let's look at Microstrategy and Telsa in particular) to see what relationship they have with crypto.